# 📝 Task 2 – Text Embedding & Semantic Index Construction  
📘 Version: 2025-07-08  

Foundational vectorization and semantic indexing of complaint narratives for **CrediTrust Financial’s Intelligent Complaint Analysis Initiative**. This notebook transforms cleaned consumer complaints into numerical embeddings using pre-trained transformer models and builds a scalable FAISS index to enable semantic search and Retrieval-Augmented Generation (RAG). Outputs from this task will power intelligent querying and automated summarization in Task 3.

---

**Challenge:** B5W6 – Intelligent Complaint Analysis  
**Company:** CrediTrust Financial  
**Author:** Nabil Mohamed  
**Branch:** `task-2-embedding-and-indexing`  
**Date:** July 2025  

---

### 📌 This notebook covers:
- Loading the cleaned complaint dataset (`filtered_complaints.csv`)
- Minimal text preprocessing for embedding readiness
- Generating sentence embeddings using transformer models (`all-MiniLM-L6-v2`)
- Constructing a FAISS index for efficient semantic search
- Running embedding quality diagnostics (dimensionality, clustering, similarity checks)
- Saving embedding artifacts and index for downstream RAG applications


In [1]:
# ------------------------------------------------------------------------------
# 🛠 Ensure Notebook Runs from Project Root (for src/ imports to work)
# ------------------------------------------------------------------------------

import os
import sys

# If running from /notebooks/, move up to project root
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
    print("📂 Changed working directory to project root")

# Add project root to sys.path so `src/` modules can be imported
project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"✅ Added to sys.path: {project_root}")

# Optional: verify file presence to confirm we're in the right place
expected_path = "data/raw"
print(
    "📁 Output path ready"
    if os.path.exists(expected_path)
    else f"⚠️ Output path not found: {expected_path}"
)

📂 Changed working directory to project root
✅ Added to sys.path: c:\Users\admin\Documents\GIT Repositories\b5-w6-intelligent-complaint-analysis-challenge
📁 Output path ready


## 📦 Imports & Environment Setup

This cell loads the core libraries required for text embedding, semantic indexing, and exploratory visualization in the context of intelligent complaint analysis. Imports are grouped by function:

- **Data handling:** `pandas`, `numpy`
- **Visualization:** `matplotlib`, `seaborn`
- **Text processing & embedding:** `re`, `nltk`, `sentence_transformers`
- **Semantic search:** `faiss`
- **Date/time analysis:** `datetime`, `pandas.to_datetime`
- **System & utilities:** `os`, `warnings`, `pathlib`


In [2]:
# ---------------------------
# 📦 Imports & Environment Setup
# ---------------------------

# Data handling
import pandas as pd  # For structured data manipulation
import numpy as np  # For numerical computations and array operations

# Visualization
import matplotlib.pyplot as plt  # For plotting visualizations
import seaborn as sns  # For enhanced plotting aesthetics

# Text processing & embedding
import re  # For regular expression–based text cleaning
import nltk  # For optional stopword removal and tokenization
from sentence_transformers import SentenceTransformer  # For pre-trained transformer embeddings

# Semantic search (vector indexing)
import faiss  # For efficient similarity search via FAISS index

# Date/time analysis
from datetime import datetime  # For date parsing and formatting
from pandas.api.types import CategoricalDtype  # For categorical data management

# System & utilities
import os  # For directory and file path operations
import warnings  # To suppress unnecessary warnings
from pathlib import Path  # For robust path handling

# Configure display settings for clarity
pd.set_option("display.max_columns", None)  # Ensure all columns are visible
pd.set_option("display.float_format", "{:,.2f}".format)  # Set consistent float formatting
warnings.filterwarnings("ignore")  # Suppress common library warnings

# Apply seaborn visual theme
sns.set(style="whitegrid", context="notebook")  # Set clean plotting style for diagnostics


c:\Users\admin\Documents\GIT Repositories\b5-w6-intelligent-complaint-analysis-challenge\complaint-analysis-challenge\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 📥 Load & Preview Cleaned Complaint Dataset (Task 2 Embedding)

This step loads the **pre-cleaned complaint dataset** (`filtered_complaints.csv`), which contains customer-submitted complaint narratives related to financial products, prepared in Task 1 through text cleaning and filtering.

- Reads the cleaned file from `data/interim/filtered_complaints.csv` using the `ComplaintChunkProcessor` module
- Verifies data structure: row and column count, presence of narrative text, and data types
- Outputs key diagnostics: dataset shape, missing values, and sample preview
- Includes defensive error handling for missing files, empty datasets, or structural inconsistencies
- Designed for robustness, reusability, and seamless integration with downstream embedding and semantic indexing tasks

This ensures that the complaint narratives are correctly staged and validated for embedding generation, FAISS indexing, and retrieval-augmented generation (RAG) in Task 3.


In [3]:
# ------------------------------------------------------------------------------
# 📥 Load Pre-Cleaned Complaint Dataset Using ComplaintChunkProcessor (Task 2)
# ------------------------------------------------------------------------------

from src.data_loader import ComplaintChunkProcessor  # Import loader class

# ✅ 1. Define path to pre-cleaned CSV (from Task 1 output)
cleaned_data_path = "data/interim/filtered_complaints.csv"  # Path to cleaned file

# ✅ 2. Initialize the processor (output_path is required but unused in this task)
processor = ComplaintChunkProcessor(
    filepath=cleaned_data_path,  # Input file path
    output_path="data/interim/dummy_output.csv",  # Dummy value for compatibility
)

# ✅ 3. Load cleaned data safely using new method
df = processor.load_cleaned_data()  # Load the cleaned complaint dataset

# ✅ 4. Quick sanity check on loaded data
if not df.empty:
    print(f"✅ Dataset loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
    display(df.head())  # Optional: Show first few rows

    # ✅ 5. Randomly sample 25,000 rows to reduce processing time (with reproducibility)
    df = df.sample(n=25_000, random_state=42).reset_index(drop=True)  # Random sample
    print(f"✅ Sampled down to: {df.shape[0]:,} rows for faster embedding")

else:
    print("⚠️ No data loaded. Check file path or contents.")

✅ Cleaned complaint dataset loaded successfully: 647,245 rows × 19 columns
✅ Dataset loaded: 647,245 rows × 19 columns


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,MappedProduct
0,2025-06-13,Credit card,Store credit card,Getting a credit card,Card opened without my consent or knowledge,a xxxx xxxx card was opened under my name by a...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",TX,78230,Servicemember,Consent provided,Web,2025-06-13,Closed with non-monetary relief,Yes,NaN,14069121,Credit card
1,2025-06-13,Checking or savings account,Checking account,Managing an account,Deposits and withdrawals,i made the mistake of using my wellsfargo debi...,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,ID,83815,NaN,Consent provided,Web,2025-06-13,Closed with explanation,Yes,NaN,14061897,Savings account
2,2025-06-12,Credit card,General-purpose credit card or charge card,"Other features, terms, or problems",Other problem,"dear cfpb, i have a secured credit card with c...",Company has responded to the consumer and the ...,"CITIBANK, N.A.",NY,11220,NaN,Consent provided,Web,2025-06-13,Closed with monetary relief,Yes,NaN,14047085,Credit card
3,2025-06-12,Credit card,General-purpose credit card or charge card,Incorrect information on your report,Account information incorrect,i have a citi rewards cards. the credit balanc...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",IL,60067,NaN,Consent provided,Web,2025-06-12,Closed with explanation,Yes,NaN,14040217,Credit card
4,2025-06-11,Vehicle loan or lease,Loan,Repossession,Deficiency balance after repossession,was never notified of repossession. once repos...,Company has responded to the consumer and the ...,CREDIT ACCEPTANCE CORPORATION,TX,75070,NaN,Consent provided,Web,2025-06-11,Closed with explanation,Yes,NaN,14019199,Personal loan


✅ Sampled down to: 25,000 rows for faster embedding


## ✨ Minimal Text Preprocessing for Embedding Readiness (Task 2)

This step applies **lossless, minimal text cleaning** to the complaint narratives to ensure readiness for sentence embedding generation. The goal is to retain the full semantic content while removing superficial noise.

- Lowercases text to ensure case consistency
- Removes excessive whitespace, line breaks, and tabs
- Preserves all other textual features to maximize embedding quality

The `MinimalTextPreprocessor` class from `src/preprocessing/minimal_text_preprocessor.py` is used for this operation, following defensive programming practices to handle missing or malformed inputs.

This ensures that embeddings are generated from standardized, clean narratives while retaining the original complaint meaning.


In [4]:
# ------------------------------------------------------------------------------
# ✨ Apply Minimal Text Preprocessing for Embedding Readiness (Task 2)
# ------------------------------------------------------------------------------

# ✅ Import the MinimalTextPreprocessor class from the preprocessing module
from src.chunking.text_cleaner import (
    MinimalTextPreprocessor,
)  # Minimal complaint cleaner for embeddings

# ✅ Initialize the preprocessor (no parameters required for minimal cleaning)
preprocessor = MinimalTextPreprocessor()  # Create preprocessor instance

# ✅ Apply text cleaning to the 'Consumer complaint narrative' column from Task 1, create new 'cleaned_narrative' column
df = preprocessor.apply_to_dataframe(
    df,  # DataFrame loaded via ComplaintChunkProcessor
    text_column="Consumer complaint narrative",  # Source text column (Task 1 cleaned)
    new_column_name="cleaned_narrative",  # Destination cleaned text column
)

# ✅ Quick manual verification: compare original and cleaned narratives side by side
df[["Consumer complaint narrative", "cleaned_narrative"]].sample(
    5, random_state=42
)  # Display 5 sample rows

,Consumer complaint narrative,cleaned_narrative
6868,on xxxxxxxx in the earlier part of the day i w...,on xxxxxxxx in the earlier part of the day i w...
24016,i received an email upon waking up on xxxxxxxx...,i received an email upon waking up on xxxxxxxx...
9668,midland credit management xxxx. xxxx xxxx xxxx...,midland credit management xxxx. xxxx xxxx xxxx...
13640,"to whom it may concern, on xxxx xxxx xxxx xxxx...","to whom it may concern, on xxxx xxxx xxxx xxxx..."
14018,"during xxxx and xxxx, i had a checking account...","during xxxx and xxxx, i had a checking account..."


## ✂️ Split Complaint Narratives into Chunks for Embedding (Task 2)

To improve the quality of semantic search and downstream retrieval, this step breaks down long complaint narratives into **smaller, semantically meaningful text chunks** using the `TextChunker` class.

Key features:
- Uses **LangChain's RecursiveCharacterTextSplitter** for intelligent chunking
- Preserves important metadata (Complaint ID, Product) for traceability
- Outputs a structured list of text chunks, each linked to its source complaint

This ensures that long complaints are represented by multiple focused embeddings, enhancing the performance of semantic search and Retrieval-Augmented Generation (RAG) pipelines.


In [5]:
# ------------------------------------------------------------------------------
# ✂️ Split Complaint Narratives into Chunks for Embedding Readiness (Task 2)
# ------------------------------------------------------------------------------

# ✅ Import the TextChunker class from the chunking module
from src.chunking.text_chunker import (
    TextChunker,
)  # Intelligent text splitter for long narratives

# ✅ 1. Initialize the chunker with desired chunk size and overlap
chunker = TextChunker(
    chunk_size=500, chunk_overlap=50
)  # Adjustable based on text length and embedding model limits

# ✅ 2. Define metadata columns to preserve for each chunk
metadata_fields = [
    "Complaint ID",
    "Product",
]  # These fields will remain attached to each chunk

# ✅ 3. Apply chunking to the cleaned narratives
chunked_data = chunker.chunk_dataframe(
    df,  # DataFrame containing cleaned narratives
    text_column="cleaned_narrative",  # Source text column to split
    metadata_columns=metadata_fields,  # Metadata to carry forward
)

# ✅ 4. Quick check: show first few chunk records with metadata
print(f"✅ Total chunks created: {len(chunked_data):,}")
pd.DataFrame(chunked_data).head()  # Preview chunks for verification

✅ Total chunks created: 73,597


,text,Complaint ID,Product,chunk_index
0,initial purchase with synchrony xxxx days late...,2681960,Credit card or prepaid card,0
1,credit account back and remove my sister off t...,2681960,Credit card or prepaid card,1
2,me 30 days late!! i did not provide documents ...,2681960,Credit card or prepaid card,2
3,from the last week of xxxx2024 to the second w...,8769778,"Money transfer, virtual currency, or money ser...",0
4,"on xxxxxxxx, i sent a bill pay check on a rela...",6217611,Checking or savings account,0


## 🔗 Generate Sentence Embeddings for Complaint Chunks (Task 2)

In this step, we transform each text chunk into a dense vector representation using the **EmbeddingGenerator** class and the pre-trained `all-MiniLM-L6-v2` transformer model.

Key features:
- Uses **GPU acceleration** when available
- Processes chunks in memory-efficient **batches**
- Embeddings are **normalized** for compatibility with cosine similarity in semantic search

This ensures that each chunk is mapped into vector space for downstream indexing and retrieval.


In [7]:
# ------------------------------------------------------------------------------
# 🔗 Generate Sentence Embeddings for Complaint Chunks Using EmbeddingGenerator
# ------------------------------------------------------------------------------

# ✅ Import the EmbeddingGenerator class from the chunking module
from src.chunking.embedding_generator import (
    EmbeddingGenerator,
)  # Modular embedding generator

# ✅ 1. Initialize the embedding generator with desired model and batch size
embedder = EmbeddingGenerator(
    model_name="all-MiniLM-L6-v2", batch_size=64
)  # Efficient transformer model

# ✅ 2. Extract text chunks from previously generated chunked_data (list of dicts)
chunk_texts = [
    record["text"] for record in chunked_data
]  # Extract chunk texts for embedding

# ✅ 3. Generate embeddings in batches with progress bar
embeddings = embedder.generate_embeddings(
    chunk_texts
)  # Generate dense vector representations

# ✅ 4. Quick sanity check: confirm embedding shape and first vector
if embeddings is not None:
    print(f"✅ Embedding matrix shape: {embeddings.shape}")
else:
    print("⚠️ No embeddings generated. Check input or model setup.")

✅ Embedding model 'all-MiniLM-L6-v2' loaded on device: cpu


Batches: 100%|██████████| 1150/1150 [1:11:38<00:00,  3.74s/it]


✅ Embedding generation successful: 73,597 vectors of dimension 384
✅ Embedding matrix shape: (73597, 384)


## 🗂️ Build ChromaDB Vector Store Using Precomputed Embeddings (Task 2)

In this step, we use the **VectorStoreBuilder** class to store precomputed complaint chunk embeddings and their associated metadata into a **ChromaDB** vector store for efficient semantic search.

Key features:
- Avoids recomputing embeddings
- Persists the index to disk for downstream RAG and interactive querying (Task 3 & 4)
- Keeps chunk traceability via metadata (Complaint ID, Product, Chunk Index)


In [21]:
# ------------------------------------------------------------------------------
# 🔄 Reload VectorStoreBuilder Module in Notebook (Jupyter/IPython)
# ------------------------------------------------------------------------------

import importlib  # Built-in module for dynamic import reloading

# ✅ Import the module (your updated code must be saved first)
import src.chunking.vector_store_builder as vector_store_builder_module

# ✅ Reload the module to get the latest changes
importlib.reload(vector_store_builder_module)

# ✅ Access the class again (fresh copy)
VectorStoreBuilder = vector_store_builder_module.VectorStoreBuilder

In [ ]:
# ------------------------------------------------------------------------------
# 🗂️ Build ChromaDB Vector Store with Progress, Diagnostics, and Runtime Insights (Task 2 – Final v2)
# ------------------------------------------------------------------------------
# Author: Nabil Mohamed | July 2025

# ✅ Standard Library Imports
import time
import numpy as np
from tqdm import tqdm

# ✅ Project Imports
from src.chunking.vector_store_builder import VectorStoreBuilder
from langchain.embeddings import HuggingFaceEmbeddings

# ------------------------------------------------------------------------------
# 1️⃣ Initialize VectorStoreBuilder
# ------------------------------------------------------------------------------

builder = VectorStoreBuilder(
    persist_directory="vector_store/chroma_db",
    collection_name="complaint_chunks",
)

# ------------------------------------------------------------------------------
# 2️⃣ Prepare Inputs
# ------------------------------------------------------------------------------

documents = chunk_texts  # List of text chunks
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

metadatas = [
    {
        "Complaint ID": chunk.get("Complaint ID", "N/A"),
        "Product": chunk.get("Product", "N/A"),
        "Chunk Index": chunk.get("chunk_index", 0),
    }
    for chunk in chunked_data
]

# Precomputed embeddings (None if not available)
precomputed_embeddings = embeddings

# ------------------------------------------------------------------------------
# 3️⃣ Diagnostics
# ------------------------------------------------------------------------------

print(f"🔍 Number of text chunks: {len(documents):,}")
print(f"🔍 Number of metadata entries: {len(metadatas):,}")

if precomputed_embeddings is not None:
    print(
        f"🔍 Precomputed embeddings detected: shape = {np.array(precomputed_embeddings).shape}"
    )
else:
    print(
        "⚙️ No precomputed embeddings provided. Will use live embedding as fallback if needed."
    )

# ------------------------------------------------------------------------------
# 4️⃣ Build Vector Store with Progress
# ------------------------------------------------------------------------------

print("\n🚀 Starting ChromaDB vector store creation...")
start_time = time.time()

with tqdm(
    total=1,
    desc="🔄 Building Vector Store",
    bar_format="{l_bar}{bar} [ time left: {remaining} ]",
) as pbar:
    vector_store = builder.build_chroma_store(
        documents=documents,
        embedding_model=embedding_model,
        metadatas=metadatas,
        embeddings=precomputed_embeddings,
    )
    pbar.update(1)

elapsed_time = time.time() - start_time

# ------------------------------------------------------------------------------
# 5️⃣ Results Summary
# ------------------------------------------------------------------------------

if vector_store:
    print(f"\n✅ ChromaDB vector store created successfully.")
    print(f"📄 Total documents indexed: {len(documents):,}")
    print(f"⏱️ Time taken: {elapsed_time:.2f} seconds")
    print(f"📁 Location: {builder.persist_directory}")
else:
    print(f"\n❌ Vector store creation failed.")
    print(f"⏱️ Time elapsed: {elapsed_time:.2f} seconds")

🔍 Number of text chunks: 73,597
🔍 Number of metadata entries: 73,597
🔍 Precomputed embeddings detected: shape = (73597, 384)

🚀 Starting ChromaDB vector store creation...


🔄 Building Vector Store:   0%|           [ time left: ? ]

⚡ Using precomputed embeddings for ChromaDB creation (Option B).
➕ Adding batch 1 (5,000 documents)...
➕ Adding batch 2 (5,000 documents)...
➕ Adding batch 3 (5,000 documents)...
➕ Adding batch 4 (5,000 documents)...
➕ Adding batch 5 (5,000 documents)...
➕ Adding batch 6 (5,000 documents)...
➕ Adding batch 7 (5,000 documents)...
➕ Adding batch 8 (5,000 documents)...
➕ Adding batch 9 (5,000 documents)...
➕ Adding batch 10 (5,000 documents)...
➕ Adding batch 11 (5,000 documents)...
➕ Adding batch 12 (5,000 documents)...
➕ Adding batch 13 (5,000 documents)...
➕ Adding batch 14 (5,000 documents)...
➕ Adding batch 15 (3,597 documents)...


🔄 Building Vector Store: 100%|██████████ [ time left: 00:00 ]

✅ ChromaDB vector store created successfully with 73,597 documents.
📁 Location: vector_store/chroma_db

✅ ChromaDB vector store created successfully.
📄 Total documents indexed: 73,597
⏱️ Time taken: 4055.63 seconds
📁 Location: vector_store/chroma_db
